In [1]:
import pandas as pd
import glob

In [2]:
# in order to use the combinator, make sure to select the correct data report file path. Although the data report exports a CSV, we have used excel formatting to more clearly highlight 
# what we are going
# the other csv files should be in the same folder

In [3]:
data_report_file_path = "data report.xlsx"
source_file_paths = glob.glob('*.csv')
data_report_df = pd.read_excel(data_report_file_path, dtype=str, na_filter=False)

In [4]:
class SourceFile:
    def __init__(self, source_filename="", source_columns = None):
        if source_columns is None:
            source_columns = []
            
        self.source_filename = source_filename
        self.source_columns =  source_columns

class SourceColumn:
    def __init__(self, column_name="",merge_type="", merge_destination='',  append_columns = None, series = None):
        if append_columns is None:
            append_columns = {}
            
        self.column_name = column_name
        self.merge_type = merge_type
        self.merge_destination = merge_destination
        self.append_columns = append_columns
        self.series = series
        
class NewFile:
    def __init__(self, new_filename="", df = pd.DataFrame()):

        self.new_filename = new_filename
        self.df =  df
        
    

In [5]:
required_files = data_report_df['File'].unique()

In [6]:
source_files = []
# creates objects required
for link in required_files:
    data_report_df_subset = data_report_df[data_report_df['File'] == link]
    df = pd.read_csv(link, na_filter=False)
    source_file = SourceFile(source_filename=link)
    source_files.append(source_file)
    for i,row in data_report_df_subset.iterrows():
        column_name = row['Column']
        merge_type = row['Data Mod']
        series = df[column_name]
        source_column = SourceColumn(column_name = column_name, merge_type = merge_type, series = series)
        source_file.source_columns.append(source_column)
        
        if row['Data Table'] and row['Data Field']:
            source_column.append_columns[row['Data Table']] = row['Data Field']
            if merge_type:
                source_column.merge_destination = row['Data Field']
            
        for j, cell in row.iteritems():
            if j.startswith('Mod') and cell:
                mod_number = j.split(' ')[1]
                mod_table_column = f'Table {mod_number}'
                mod_field_column = f'Field {mod_number}'
                if cell == 'Append':
                    source_column.append_columns[row[mod_table_column]]=row[mod_field_column]


In [7]:
def merge_source_column(base_source_column, additional_source_column):
    if ',' in base_source_column.merge_type:
        merge_delimiter = ','
    elif ';' in base_source_column.merge_type:
        merge_delimiter = ';'
    else:
        # is merge delimiter is None, do priority merge
        merge_delimiter = None

    if merge_delimiter:
        base_source_column.series = base_source_column.series + merge_delimiter + additional_source_column.series
    else:
        base_source_column.series = base_source_column.series.where(base_source_column.series != '', additional_source_column.series)
    additional_source_column.append_columns = {}


In [8]:
for source_file in source_files:
    columns_to_merge = {}
    # create columns to merge
    for source_column in source_file.source_columns:
        merge_destination = source_column.merge_destination
        if merge_destination == '':
            continue
        if merge_destination in columns_to_merge:
            columns_to_merge[merge_destination].append(source_column)
        else:
            columns_to_merge[merge_destination] = [source_column]
    # orders the merges then performs them
    for destination, source_column_list in columns_to_merge.items():
        source_column_list = sorted(source_column_list, key=lambda x: x.merge_type)
        for i, source_column in enumerate(source_column_list):
            if i > 0:
                base_source_column = source_column_list[0]
                merge_source_column(base_source_column, source_column)
            

In [9]:
# sets up append
new_files = {}
for source_file in source_files:
    new_files_to_series = {}
    for source_column in source_file.source_columns:
        for new_file, new_column in source_column.append_columns.items():
            if new_file in new_files_to_series:
                new_files_to_series[new_file].append(source_column)
            else:
                new_files_to_series[new_file] = [source_column]
    # appends done here
    for new_file_name, source_columns in new_files_to_series.items():
        if new_file_name not in new_files:
            new_files[new_file_name] = NewFile(new_filename=new_file_name)
            new_file = new_files[new_file_name]
        else:
            new_file =new_files[new_file_name]
            
        dataframe_dict = {}
        for source_column in source_columns:
            dataframe_dict[source_column.append_columns[new_file_name]] = list(source_column.series)
        
        df_subset = pd.DataFrame(dataframe_dict)
        df_subset.insert(0,'Source File', source_file.source_filename)
        new_file.df = pd.concat([new_file.df, df_subset], sort=False, ignore_index=True)
        new_file.df.astype(int, errors='ignore')


In [10]:
for filename, new_file in new_files.items():
    df = new_file.df
    df.to_csv(f'{filename}.csv')


In [11]:
# this is unrelated to the functionality of the combinator but we needed this to generate the analysis

In [12]:
group_df = new_file.df.groupby('Zip').sum()

In [13]:
group_df.to_csv(f'{filename}2.csv')
df = pd.read_csv('visits2.csv')

In [14]:
df = df.groupby('Zip').sum()
df.to_csv(f'{filename}2.csv')
